# OAI RAN Bug Root-Cause Analysis Workflow

This notebook implements a structured workflow for investigating bugs in the OpenAirInterface5G (OAI) RAN project. It follows the process defined in `QAprompt.json`, guiding the user from raw bug reports to root-cause analysis by cross-referencing specifications and code.

### Agent Instruction

If you are a AI coding agent, you are given access to access the entire codebase in the current folder. Also, you have access to "spec_ingestor.py" to help you make correct citation to the latest 3GPP standard. Note it is not using AI and you should consider it a grep tool with structured output. If you make any claim about 3GPP standard, you *MUST* verify it with this tool. Use it strictly follow this pattern "C:/Users/woaiw/cmd/envs/CS211/python.exe p:/openairinterface5g/openair2/spec_ingestor.py F1AP.docx 'keyword'"

### Global Definitions
The workflow adheres to the following definitions and enumerations:

```json
  "$defs": {
    "RepoEnum": {
      "type": "string",
      "enum": ["https://gitlab.eurecom.fr/oai/openairinterface5g/-/tree/develop/openair2/F1AP?ref_type=heads"]
    },
    "InterfaceEnum": {
      "type": "string",
      "enum": ["F1AP","E1AP","NGAP","XnAP","RRC","PDCP","RLC","MAC","GTP-U","SCTP"]
    },
    "VerdictEnum": {
      "type": "string",
      "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"]
    },
    "ComponentEnum": {
      "type": "string",
      "enum": ["gNB-CU", "gNB-DU", "UE", "AMF", "UPF"]
    }
  },
```

The workflow consists of sequential steps, each broken down into:
1. **Requirements:** Definition of the step's goal, inputs, and expected outputs based on `QAprompt.json`.
2. **Results:** Placeholders for the execution results.
3. **Check:** Verification criteria to ensure the step was completed successfully.

## Step 1: Bug Ingest

### 1.1 Requirements

**Goal:** Normalize a freeform bug report into a structured bug card with interface/procedure guesses and key identifiers.

**Input:**
- `bug_text`: Bug: gNB-DU crashes when receiving DLRRCMessage Transfer after releasing UE.
Repro Steps:
1) The attacker intercepts a legitimate UE Context Modification request.
2) Modify the RNTI in the IE to an incorrect value and forward it to the gNB-CU.
3) For gNB-DUs, the attacker waits for UE release and immediately sends a DLRRCMessage Transfer.

**Role & Instructions:**
Act as **BugCardBuilder**. Extract and normalize from `bug_text`: likely interface(s), procedure(s), component roles, and key IDs (transaction IDs, CU/DU UE F1AP IDs, DU ID, RNTI, PCI, served-cell list). Summarize observed vs expected (if implied).

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["bug_card"],
        "properties": {
          "bug_card": {
            "type": "object",
            "required": ["observed_behavior", "interface_guess", "procedure_guess", "components_involved"],
            "properties": {
              "observed_behavior": { "type": "string" },
              "interface_guess": { "type": "array", "items": {"$ref":"#/$defs/InterfaceEnum" } },
              "procedure_guess": { "type": "array", "items": { "type": "string" } },
              "components_involved": { "type": "array", "items": {"$ref":"#/$defs/ComponentEnum" },"minItems":1 },
              "key_ids": {
                "type": "object",
                "properties": {
                  "transaction_id": { "type": "string" },
                  "cu_ue_f1ap_id": { "type": "string" },
                  "du_ue_f1ap_id": { "type": "string" },
                  "du_id": { "type": "string" },
                  "rnti": { "type": "string" },
                  "pci": { "type": "string" },
                  "served_cell_list": { "type": "array", "items": { "type": "string" } }
                }
              },
              "signals_or_timers": { "type": "array", "items": { "type": "string" } }
            }
          }
        }
      }
```

### 1.2 Results

```json
{
  "bug_card": {
    "observed_behavior": "gNB-DU crashes when receiving DL RRC MESSAGE TRANSFER after UE context was released.",
    "interface_guess": ["F1AP"],
    "procedure_guess": ["UE Context Release", "DL RRC Message Transfer", "UE Context Modification"],
    "components_involved": ["gNB-CU", "gNB-DU", "UE"],
    "key_ids": {
      "transaction_id": "",
      "cu_ue_f1ap_id": "",
      "du_ue_f1ap_id": "",
      "du_id": "",
      "rnti": "tampered/incorrect",
      "pci": "",
      "served_cell_list": []
    },
    "signals_or_timers": ["gNB-CU UE F1AP ID", "gNB-DU UE F1AP ID"]
  }
}
```

### 1.3 Check

**Verification:** Check that the output matches the JSON schema, specifically that `interface_guess` and `procedure_guess` are not empty, and no freeform prose is included.

## Step 2: Spec Section Fetcher

### 2.1 Requirements

**Goal:** Propose spec sections (procedures, timers, message formats) that likely govern the bug.

**Input:**
- `bug_card`: Structured description from Step 1.
- `spec_toc`: Table of contents with ids/titles/anchors.

**Role & Instructions:**
Act as **SpecIndexer**. Using `bug_card` and attached spec slices, identify 3–6 most relevant spec sections. Produce keywords, titles, and summarize `expected_behaviour`.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_spec_sections","expected_behaviour"],
        "properties": {
          "candidate_spec_sections": {
            "type": "array",
            "minItems": 3,
            "maxItems": 6,
            "items": {
              "type": "object",
              "required": ["Keywords", "title"],
              "properties": { "Keywords": { "type": "array", "items": { "type": "string" }}, "title": { "type": "array", "items": { "type": "string" }} }
            }
          },
          "expected_behaviour": {"type": "string"}
        }
      }
```

### 2.2 Results

```json
{
  "candidate_spec_sections": [
    {"Keywords": ["DL RRC Message Transfer", "UE-associated", "ID binding"], "title": ["§8.4.2 DL RRC Message Transfer", "§9.2.3.2 DL RRC MESSAGE TRANSFER"]},
    {"Keywords": ["UE Context Release", "release resources", "Complete"], "title": ["§8.3.3 UE Context Release", "§9.2.2.5 UE CONTEXT RELEASE COMMAND", "§9.2.2.6 UE CONTEXT RELEASE COMPLETE"]},
    {"Keywords": ["Identifier", "gNB-CU UE F1AP ID", "gNB-DU UE F1AP ID"], "title": ["§9.3.1.4 gNB-CU UE F1AP ID", "§9.3.1.5 gNB-DU UE F1AP ID"]},
    {"Keywords": ["UE Context Release Request", "cause values"], "title": ["§8.3.2 UE Context Release Request", "§9.2.2.4 UE CONTEXT RELEASE REQUEST"]}
  ],
  "expected_behaviour": "CU-initiated release: DU releases UE resources and completes. Later DL RRC MESSAGE TRANSFER must reference valid UE IDs; if no UE context exists, DU shall not crash, may establish UE-associated F1 connection or trigger Release Request per spec."
}
```

### 2.3 Check

**Verification:** Check that 3-6 relevant sections are identified and `expected_behaviour` is summarized.

## Step 3: Code Fetcher (metadata-driven)

### 3.1 Requirements

**Goal:** Select likely source files/functions using metadata (paths, interface names, keywords, ASN.1 types, timers).

**Input:**
- `bug_card`: From Step 1.
- `candidate_spec_sections`: From Step 2.
- `repo_metadata`: Path, interface names, and known keywords.

**Role & Instructions:**
Act as **CodeLocator**. Build a prioritized list of files/functions to fetch using interface names, procedure/timer, and keywords. Plan recursion if initial fetch seems insufficient.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_code"],
        "properties": {
          "candidate_code": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["path", "function_name", "reason"],
              "properties": { "path": { "type": "string" }, "function_name": { "type": "string" }, "reason": { "type": "string" } }
            }
          }
        }
      }
```

### 3.2 Results

```json
{
  "candidate_code": [
    {"path": "F1AP/f1ap_du_rrc_message_transfer.c", "function_name": "DU_handle_DL_RRC_MESSAGE_TRANSFER", "reason": "DU entry handler for DL RRC MESSAGE TRANSFER."},
    {"path": "LAYER2/NR_MAC_gNB/mac_rrc_dl_handler.c", "function_name": "dl_rrc_message_transfer", "reason": "Checks UE existence, maps IDs, forwards to RLC; defines behavior if UE absent."},
    {"path": "F1AP/lib/f1ap_rrc_message_transfer.c", "function_name": "decode_dl_rrc_message_transfer", "reason": "Decodes CU/DU UE IDs and RRC container; error/ownership paths."},
    {"path": "LAYER2/NR_MAC_gNB/mac_rrc_dl_handler.c", "function_name": "ue_context_release_command", "reason": "DU handling of CU-initiated UE release; post-release state."},
    {"path": "F1AP/lib/f1ap_ue_context.c", "function_name": "decode_ue_context_rel_cmd", "reason": "Decoding UE CONTEXT RELEASE COMMAND IDs and cause."},
    {"path": "F1AP/f1ap_handlers.c", "function_name": "DU_handle_DL_RRC_MESSAGE_TRANSFER (dispatch)", "reason": "Message dispatch mapping for DLRRCMessageTransfer."}
  ]
}
```

### 3.3 Check

**Verification:** Check that `candidate_code` is not empty and includes paths and function names.

## Step 4: Lightweight Retriever

### 4.1 Requirements

**Goal:** Retrieve high-signal snippets only from shortlisted specs and code.

**Input:**
- `candidate_spec_sections`: From Step 2.
- `candidate_code`: From Step 3.

**Role & Instructions:**
Act as **SnippetRetriever**. For each query, return top snippets with exact locations (spec id/anchor, file:line-range). Keep each snippet ≤ 120 words.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["snippets"],
        "properties": {
          "snippets": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["source", "location", "text", "kind"],
              "properties": {
                "kind": { "type": "string", "enum": ["spec", "code"] },
                "source": { "type": "string" },
                "location": { "type": "string" },
                "text": { "type": "string" }
              }
            }
          }
        }
      }
```

### 4.2 Results

```json
{
  "snippets": [
    {"kind": "spec", "source": "F1AP.docx", "location": "§8.4.2.2", "text": "The gNB-CU initiates DL RRC MESSAGE TRANSFER. If a UE-associated F1 connection exists, the message shall contain the gNB-DU UE F1AP ID used to lookup the UE context. If no UE-associated logical F1-connection exists, it shall be established at reception. If UE Context not retrievable is true, DU may trigger UE Context Release Request."},
    {"kind": "spec", "source": "F1AP.docx", "location": "§8.3.3.2", "text": "Upon reception of UE CONTEXT RELEASE COMMAND, the gNB-DU shall release all related signalling and user data resources and reply with UE CONTEXT RELEASE COMPLETE. If old gNB-DU UE F1AP ID is included, also release that context."},
    {"kind": "spec", "source": "F1AP.docx", "location": "§9.3.1.4/§9.3.1.5", "text": "gNB-CU UE F1AP ID and gNB-DU UE F1AP ID uniquely identify the UE association within CU and DU respectively, and are mandatory in DL RRC MESSAGE TRANSFER."},
    {"kind": "code", "source": "openair2/F1AP/f1ap_du_rrc_message_transfer.c", "location": "58-70", "text": "int DU_handle_DL_RRC_MESSAGE_TRANSFER(...){ f1ap_dl_rrc_message_t msg={0}; if(!decode_dl_rrc_message_transfer(pdu,&msg)){ LOG_E(...); free_dl_rrc_message_transfer(&msg); return -1; } dl_rrc_message_transfer(&msg); free_dl_rrc_message_transfer(&msg); return 0; }"},
    {"kind": "code", "source": "openair2/LAYER2/NR_MAC_gNB/mac_rrc_dl_handler.c", "location": "918-935", "text": "dl_rrc_message_transfer(const f1ap_dl_rrc_message_t *dl_rrc){ ... NR_UE_info_t *UE = find_nr_UE(..., dl_rrc->gNB_DU_ue_id); UE = UE ? UE : find_ra_UE(...); if(UE==NULL){ LOG_E(..., \"unknown UE ...\"); return; } ... }"},
    {"kind": "code", "source": "openair2/F1AP/lib/f1ap_rrc_message_transfer.c", "location": "396-404", "text": "F1AP_DLRRCMessageTransfer_t *container = &pdu->choice.initiatingMessage->value.choice.DLRRCMessageTransfer; ... F1AP_LIB_FIND_IE(..., id_gNB_CU_UE_F1AP_ID, true); out->gNB_CU_ue_id = ...; F1AP_LIB_FIND_IE(..., id_gNB_DU_UE_F1AP_ID, true); out->gNB_DU_ue_id = ...;"}
  ]
}
```

### 4.3 Check

**Verification:** Check that snippets are retrieved from both spec and code, with exact locations.

## Step 5: Event/State Sketcher (draft)

### 5.1 Requirements

**Goal:** Draft expected message sequence and 5–8 state sketch from snippets.

**Input:**
- `bug_card`: From Step 1.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **SketchMaker**. Build (1) an expected message sequence as ordered events and (2) a compact state machine (5–8 states, transitions with guards). Note assumptions and open questions.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["sequence_diagram", "state_machine", "assumptions", "questions", "review_request"],
        "properties": {
          "sequence_diagram": {
            "type": "array",
            "items": { "type": "object", "required": ["from", "to", "message"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "message": { "type": "string" }, "precond": { "type": "string" }, "postcond": { "type": "string" } } }
          },
          "state_machine": {
            "type": "object",
            "required": ["states", "transitions"],
            "properties": {
              "states": { "type": "array", "items": { "type": "string" }, "minItems": 5, "maxItems": 8 },
              "transitions": {
                "type": "array",
                "items": { "type": "object", "required": ["from", "to", "on"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "on": { "type": "string" }, "guard": { "type": "string" } } }
              }
            }
          },
          "assumptions": { "type": "array", "items": { "type": "string" } },
          "questions": { "type": "array", "items": { "type": "string" } },
          "review_request": { "type": "string", "enum": ["HumanInLoop_B_required"] }
        }
      }
```

### 5.2 Results

```json
{
  "sequence_diagram": [
    {"from": "gNB-CU", "to": "gNB-DU", "message": "UE CONTEXT RELEASE COMMAND", "precond": "UE-connected", "postcond": "DU releasing UE resources"},
    {"from": "gNB-DU", "to": "gNB-CU", "message": "UE CONTEXT RELEASE COMPLETE", "precond": "Release executed", "postcond": "UE context removed (or kept per CG-SDT indicator)"},
    {"from": "gNB-CU", "to": "gNB-DU", "message": "DL RRC MESSAGE TRANSFER (SRB)", "precond": "UE-associated F1 exists and IDs valid", "postcond": "DU forwards to RLC"},
    {"from": "gNB-CU(attacker)", "to": "gNB-DU", "message": "DL RRC MESSAGE TRANSFER after release (IDs may mismatch)", "precond": "UE context released", "postcond": "DU ignores/logs or re-establishes per spec; must not crash"}
  ],
  "state_machine": {
    "states": ["UE_CONNECTED", "RELEASING", "RELEASED", "REESTABLISHMENT_ONGOING", "NEW_CONN_ESTABLISHING"],
    "transitions": [
      {"from": "UE_CONNECTED", "to": "RELEASING", "on": "UE_CONTEXT_RELEASE_COMMAND", "guard": "IDs match (§9.3.1.4/§9.3.1.5)"},
      {"from": "RELEASING", "to": "RELEASED", "on": "UE_CONTEXT_RELEASE_COMPLETE"},
      {"from": "RELEASED", "to": "NEW_CONN_ESTABLISHING", "on": "DL_RRC_MESSAGE_TRANSFER", "guard": "No UE F1-conn; DU establishes per §8.4.2.2"},
      {"from": "RELEASED", "to": "RELEASED", "on": "DL_RRC_MESSAGE_TRANSFER", "guard": "IDs unknown/inconsistent; log/ignore; may trigger Release Request"},
      {"from": "UE_CONNECTED", "to": "REESTABLISHMENT_ONGOING", "on": "DL_RRC_MESSAGE_TRANSFER with old DU UE ID", "guard": "old_gNB-DU UE ID present"},
      {"from": "REESTABLISHMENT_ONGOING", "to": "UE_CONNECTED", "on": "Reconfiguration applied", "guard": "SRB1 reconfigured"}
    ]
  },
  "assumptions": [
    "Attack modifies RNTI or DU UE ID causing mismatch.",
    "DU maps DU UE ID to RNTI; missing UE causes early return (no crash).",
    "No DU-local watchdog triggers restart in this path."
  ],
  "questions": [
    "Crash stack trace location (DU F1AP vs MAC vs RLC)?",
    "Exact F1 message fields seen on wire (IDs, SRB, old DU ID)?"
  ],
  "review_request": "HumanInLoop_B_required"
}
```

### 5.3 Check

**Verification:** Check for a sequence diagram, a state machine with 5-8 states, and any assumptions.

## Step 6: Conformance Checker (simple invariants)

### 6.1 Requirements

**Goal:** Check core invariants to propose preliminary verdict.

**Input:**
- `bug_card`: From Step 1.
- `sequence_diagram`: From Step 5.
- `state_machine`: From Step 5.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **ConformanceChecker**. Using snippets as authority, test invariants: directionality, req/resp pairing, ID binding, transaction scoping, timer preconditions. For each invariant: PASS/FAIL with cite. Propose preliminary verdict.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["invariant_checks", "preliminary_verdict", "rationale"],
        "properties": {
          "invariant_checks": {
            "type": "array",
            "items": { "type": "object", "required": ["name", "result", "evidence"], "properties": { "name": { "type": "string" }, "result": { "type": "string", "enum": ["PASS", "FAIL", "UNKNOWN"] }, "evidence": { "type": "array", "items": { "type": "string" } } } }
          },
          "preliminary_verdict": { "type": "string", "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"] },
          "rationale": { "type": "string" }
        }
      }
```

### 6.2 Results

```json
{
  "invariant_checks": [
    {"name": "DL RRC contains both UE IDs (CU/DU)", "result": "PASS", "evidence": ["Spec §9.2.3.2", "Code openair2/F1AP/lib/f1ap_rrc_message_transfer.c:396-404"]},
    {"name": "DU does not process DL RRC for unknown UE", "result": "PASS", "evidence": ["Code openair2/LAYER2/NR_MAC_gNB/mac_rrc_dl_handler.c:918-935"]},
    {"name": "Release frees resources and completes", "result": "PASS", "evidence": ["Spec §8.3.3.2", "Code openair2/LAYER2/NR_MAC_gNB/mac_rrc_dl_handler.c:883-916"]},
    {"name": "Old DU UE ID triggers reestablishment mapping", "result": "PASS", "evidence": ["Spec §8.4.2.2 (old DU ID)", "Code openair2/LAYER2/NR_MAC_gNB/mac_rrc_dl_handler.c:946-988"]}
  ],
  "preliminary_verdict": "needs-more-data",
  "rationale": "Code paths check UE existence and early-return without crash on unknown UE; spec requires IDs and allows DU to establish or trigger Release Request. The reported crash likely stems from an external factor (e.g., memory ownership, logging, or watchdog) not visible here. Logs and stack trace are needed."
}
```

### 6.3 Check

**Verification:** Check that invariants are tested and a preliminary verdict is proposed.

## Step 7: Evidence Linker

### 7.1 Requirements

**Goal:** Bundle a compact evidence pack linking spec clauses and code lines with an expected-vs-observed chain.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **EvidenceBundler**. Assemble a minimal yet complete pack: spec refs (id/anchor), code refs (file:lines), and a single paragraph contrasting expected vs observed.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["evidence_pack"],
        "properties": {
          "evidence_pack": {
            "type": "object",
            "required": ["spec_citations", "code_citations", "expected_vs_observed"],
            "properties": {
              "spec_citations": { "type": "array", "items": { "type": "string" } },
              "code_citations": { "type": "array", "items": { "type": "string" } },
              "expected_vs_observed": { "type": "string" }
            }
          }
        }
      }
```

### 7.2 Results

```json
{
  "evidence_pack": {
    "spec_citations": [
      "F1AP.docx §8.4.2.2",
      "F1AP.docx §9.2.3.2",
      "F1AP.docx §8.3.3.2",
      "F1AP.docx §9.3.1.4",
      "F1AP.docx §9.3.1.5"
    ],
    "code_citations": [
      "openair2/F1AP/f1ap_du_rrc_message_transfer.c:58-70",
      "openair2/LAYER2/NR_MAC_gNB/mac_rrc_dl_handler.c:918-935",
      "openair2/LAYER2/NR_MAC_gNB/mac_rrc_dl_handler.c:883-916",
      "openair2/F1AP/lib/f1ap_rrc_message_transfer.c:396-404"
    ],
    "expected_vs_observed": "Expected: After UE CONTEXT RELEASE, DU releases resources and either ignores or properly handles subsequent DL RRC MESSAGE TRANSFER based on valid IDs; no crash. Observed: DU reportedly crashes when DL RRC MESSAGE TRANSFER arrives post-release with tampered/incorrect RNTI/ID."
  }
}
```

### 7.3 Check

**Verification:** Check that an evidence pack with spec and code citations and a comparison paragraph is created.

## Step 8: Report Generator

### 8.1 Requirements

**Goal:** Emit a one-page final report and request human sign-off on verdict.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `evidence_pack`: From Step 7.

**Role & Instructions:**
Act as **ReportWriter**. Produce the final JSON report with {verdict, rationale, citations, suggested_repro, risks, next_steps}. Be concise and actionable.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["report", "review_request"],
        "properties": {
          "report": {
            "type": "object",
            "required": ["verdict", "rationale", "citations", "suggested_repro", "next_steps"],
            "properties": {
              "verdict": { "type": "string", "$ref": "#/$defs/VerdictEnum" },
              "rationale": { "type": "string" },
              "citations": {
                "type": "object",
                "properties": {
                  "spec": { "type": "array", "items": { "type": "string" } },
                  "code": { "type": "array", "items": { "type": "string" } }
                }
              },
              "suggested_repro": { "type": "array", "items": { "type": "string" } },
              "risks": { "type": "array", "items": { "type": "string" } },
              "next_steps": { "type": "array", "items": { "type": "string" } }
            }
          },
          "review_request": { "type": "string", "enum": ["HumanInLoop_C_required"] }
        }
      }
```

### 8.2 Results

```json
{
  "report": {
    "verdict": "needs-more-data",
    "rationale": "Spec requires DU to use CU/DU UE IDs for DL RRC MESSAGE TRANSFER and to release resources on UE CONTEXT RELEASE. OAI DU code checks unknown UE and returns without processing; no obvious crash path is visible in the inspected handlers. The crash likely originates from an external subsystem or an unobserved code path; logs/stack trace are required.",
    "citations": {
      "spec": [
        "F1AP.docx §8.4.2.2",
        "F1AP.docx §9.2.3.2",
        "F1AP.docx §8.3.3.2",
        "F1AP.docx §9.3.1.4",
        "F1AP.docx §9.3.1.5"
      ],
      "code": [
        "openair2/F1AP/f1ap_du_rrc_message_transfer.c:58-70",
        "openair2/LAYER2/NR_MAC_gNB/mac_rrc_dl_handler.c:918-935",
        "openair2/LAYER2/NR_MAC_gNB/mac_rrc_dl_handler.c:883-916",
        "openair2/F1AP/lib/f1ap_rrc_message_transfer.c:396-404"
      ]
    },
    "suggested_repro": [
      "Attach DU to CU, complete UE setup.",
      "Trigger UE CONTEXT RELEASE from CU and wait for COMPLETE.",
      "Send DL RRC MESSAGE TRANSFER with mismatched/invalid gNB-DU UE F1AP ID (or tampered RNTI).",
      "Collect DU logs with log levels F1AP/NR_MAC=debug and core dumps enabled."
    ],
    "risks": [
      "Potential DU crash due to unexpected null deref in non-guarded path.",
      "Resource leak or double-free on malformed message handling."
    ],
    "next_steps": [
      "Capture DU stack trace (addr2line) for the crash; identify module.",
      "Enable ASAN/UBSAN if possible and reproduce.",
      "Add defensive checks around dl_rrc_message_transfer path; assert-free error handling.",
      "Add unit/integration test sending DL RRC after release with bad IDs."
    ]
  },
  "review_request": "HumanInLoop_C_required"
}
```

### 8.3 Check

**Verification:** Check that the final report contains a verdict, rationale, citations, and next steps.